# Düzenlileştirme (Regularization)

Derin Öğrenme modelleri o kadar fazla esnekliğe ve kapasiteye sahiptir ki, eğitim veri kümesi yeterince büyük değilse **aşırı uyum (overfitting) ciddi bir sorun olabilir**. Elbette eğitim kümesinde iyi sonuç veriyor, ancak öğrenilen ağ daha önce hiç görmediği **yeni örneklere genelleme yapmıyor**!

Öncelikle kullanacağımız paketleri import edelim.

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import h5py
#from reg_utils import sigmoid, relu, plot_decision_boundary, initialize_parameters, load_2D_dataset, predict_dec
#from reg_utils import compute_cost, predict, forward_propagation, backward_propagation, update_parameters
import sklearn
import sklearn.datasets
import sklearn.linear_model
import scipy.io
#from testCases import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import warnings
warnings.filterwarnings('ignore')

In [ ]:
ROOT_DIR = "https://raw.githubusercontent.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART2/Day9-NeuralNetworks/notebooks/"

DATASET_PATH = ROOT_DIR + "datasets/"

In [ ]:
DATASET_2D = DATASET_PATH + 'data.mat'

!wget $DATASET_2D

In [ ]:
#@title Yardımcı Fonksiyonlar
def sigmoid(x):
    """
    Compute the sigmoid of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

def relu(x):
    """
    Compute the relu of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- relu(x)
    """
    s = np.maximum(0,x)
    
    return s

def load_planar_dataset(seed):
    
    np.random.seed(seed)
    
    m = 400 # number of examples
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 4 # maximum ray of the flower

    for j in range(2):
        ix = range(N*j,N*(j+1))
        t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
        r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j
        
    X = X.T
    Y = Y.T

    return X, Y

def initialize_parameters(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    b1 -- bias vector of shape (layer_dims[l], 1)
                    Wl -- weight matrix of shape (layer_dims[l-1], layer_dims[l])
                    bl -- bias vector of shape (1, layer_dims[l])
                    
    Tips:
    - For example: the layer_dims for the "Planar Data classification model" would have been [2,2,1]. 
    This means W1's shape was (2,2), b1 was (1,2), W2 was (2,1) and b2 was (1,1). Now you have to generalize it!
    - In the for loop, use parameters['W' + str(l)] to access Wl, where l is the iterative integer.
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == layer_dims[l], layer_dims[l-1])
        assert(parameters['W' + str(l)].shape == layer_dims[l], 1)

        
    return parameters

def forward_propagation(X, parameters):
    """
    Implements the forward propagation (and computes the loss) presented in Figure 2.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape ()
                    b1 -- bias vector of shape ()
                    W2 -- weight matrix of shape ()
                    b2 -- bias vector of shape ()
                    W3 -- weight matrix of shape ()
                    b3 -- bias vector of shape ()
    
    Returns:
    loss -- the loss function (vanilla logistic loss)
    """
        
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return A3, cache

def backward_propagation(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat)
    cache -- cache output from forward_propagation()
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(i)] = Wi
                    parameters['b' + str(i)] = bi
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(i)] = dWi
                    grads['db' + str(i)] = dbi
    learning_rate -- the learning rate, scalar.
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    
    n = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for k in range(n):
        parameters["W" + str(k+1)] = parameters["W" + str(k+1)] - learning_rate * grads["dW" + str(k+1)]
        parameters["b" + str(k+1)] = parameters["b" + str(k+1)] - learning_rate * grads["db" + str(k+1)]
        
    return parameters

def predict(X, y, parameters):
    """
    This function is used to predict the results of a  n-layer neural network.
    
    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    p = np.zeros((1,m), dtype = np.int)
    
    # Forward propagation
    a3, caches = forward_propagation(X, parameters)
    
    # convert probas to 0/1 predictions
    for i in range(0, a3.shape[1]):
        if a3[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0

    # print results

    #print ("predictions: " + str(p[0,:]))
    #print ("true labels: " + str(y[0,:]))
    print("Accuracy: "  + str(np.mean((p[0,:] == y[0,:]))))
    
    return p

def compute_cost(a3, Y):
    """
    Implement the cost function
    
    Arguments:
    a3 -- post-activation, output of forward propagation
    Y -- "true" labels vector, same shape as a3
    
    Returns:
    cost - value of the cost function
    """
    m = Y.shape[1]
    
    logprobs = np.multiply(-np.log(a3),Y) + np.multiply(-np.log(1 - a3), 1 - Y)
    cost = 1./m * np.nansum(logprobs)
    
    return cost

def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    train_set_x_orig = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
    test_set_x_orig = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
    
    train_set_x = train_set_x_orig/255
    test_set_x = test_set_x_orig/255

    return train_set_x, train_set_y, test_set_x, test_set_y, classes


def predict_dec(parameters, X):
    """
    Used for plotting decision boundary.
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (m, K)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Predict using forward propagation and a classification threshold of 0.5
    a3, cache = forward_propagation(X, parameters)
    predictions = (a3>0.5)
    return predictions

def load_planar_dataset(randomness, seed):
    
    np.random.seed(seed)
    
    m = 50
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 2 # maximum ray of the flower

    for j in range(2):
        
        ix = range(N*j,N*(j+1))
        if j == 0:
            t = np.linspace(j, 4*3.1415*(j+1),N) #+ np.random.randn(N)*randomness # theta
            r = 0.3*np.square(t) + np.random.randn(N)*randomness # radius
        if j == 1:
            t = np.linspace(j, 2*3.1415*(j+1),N) #+ np.random.randn(N)*randomness # theta
            r = 0.2*np.square(t) + np.random.randn(N)*randomness # radius
            
        X[ix] = np.c_[r*np.cos(t), r*np.sin(t)]
        Y[ix] = j
        
    X = X.T
    Y = Y.T

    return X, Y

def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)
    plt.show()
    
def load_2D_dataset():
    data = scipy.io.loadmat('data.mat')
    train_X = data['X'].T
    train_Y = data['y'].T
    test_X = data['Xval'].T
    test_Y = data['yval'].T

    plt.scatter(train_X[0, :], train_X[1, :], c=train_Y, s=40, cmap=plt.cm.Spectral);
    
    return train_X, train_Y, test_X, test_Y

In [ ]:
#@title Yardımcı Test Fonksiyonları
import numpy as np

def compute_cost_with_regularization_test_case():
    np.random.seed(1)
    Y_assess = np.array([[1, 1, 0, 1, 0]])
    W1 = np.random.randn(2, 3)
    b1 = np.random.randn(2, 1)
    W2 = np.random.randn(3, 2)
    b2 = np.random.randn(3, 1)
    W3 = np.random.randn(1, 3)
    b3 = np.random.randn(1, 1)
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2, "W3": W3, "b3": b3}
    a3 = np.array([[ 0.40682402,  0.01629284,  0.16722898,  0.10118111,  0.40682402]])
    return a3, Y_assess, parameters

def backward_propagation_with_regularization_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(3, 5)
    Y_assess = np.array([[1, 1, 0, 1, 0]])
    cache = (np.array([[-1.52855314,  3.32524635,  2.13994541,  2.60700654, -0.75942115],
         [-1.98043538,  4.1600994 ,  0.79051021,  1.46493512, -0.45506242]]),
  np.array([[ 0.        ,  3.32524635,  2.13994541,  2.60700654,  0.        ],
         [ 0.        ,  4.1600994 ,  0.79051021,  1.46493512,  0.        ]]),
  np.array([[-1.09989127, -0.17242821, -0.87785842],
         [ 0.04221375,  0.58281521, -1.10061918]]),
  np.array([[ 1.14472371],
         [ 0.90159072]]),
  np.array([[ 0.53035547,  5.94892323,  2.31780174,  3.16005701,  0.53035547],
         [-0.69166075, -3.47645987, -2.25194702, -2.65416996, -0.69166075],
         [-0.39675353, -4.62285846, -2.61101729, -3.22874921, -0.39675353]]),
  np.array([[ 0.53035547,  5.94892323,  2.31780174,  3.16005701,  0.53035547],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]),
  np.array([[ 0.50249434,  0.90085595],
         [-0.68372786, -0.12289023],
         [-0.93576943, -0.26788808]]),
  np.array([[ 0.53035547],
         [-0.69166075],
         [-0.39675353]]),
  np.array([[-0.3771104 , -4.10060224, -1.60539468, -2.18416951, -0.3771104 ]]),
  np.array([[ 0.40682402,  0.01629284,  0.16722898,  0.10118111,  0.40682402]]),
  np.array([[-0.6871727 , -0.84520564, -0.67124613]]),
  np.array([[-0.0126646]]))
    return X_assess, Y_assess, cache

def forward_propagation_with_dropout_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(3, 5)
    W1 = np.random.randn(2, 3)
    b1 = np.random.randn(2, 1)
    W2 = np.random.randn(3, 2)
    b2 = np.random.randn(3, 1)
    W3 = np.random.randn(1, 3)
    b3 = np.random.randn(1, 1)
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2, "W3": W3, "b3": b3}
    
    return X_assess, parameters

def backward_propagation_with_dropout_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(3, 5)
    Y_assess = np.array([[1, 1, 0, 1, 0]])
    cache = (np.array([[-1.52855314,  3.32524635,  2.13994541,  2.60700654, -0.75942115],
           [-1.98043538,  4.1600994 ,  0.79051021,  1.46493512, -0.45506242]]), np.array([[ True, False,  True,  True,  True],
           [ True,  True,  True,  True, False]], dtype=bool), np.array([[ 0.        ,  0.        ,  4.27989081,  5.21401307,  0.        ],
           [ 0.        ,  8.32019881,  1.58102041,  2.92987024,  0.        ]]), np.array([[-1.09989127, -0.17242821, -0.87785842],
           [ 0.04221375,  0.58281521, -1.10061918]]), np.array([[ 1.14472371],
           [ 0.90159072]]), np.array([[ 0.53035547,  8.02565606,  4.10524802,  5.78975856,  0.53035547],
           [-0.69166075, -1.71413186, -3.81223329, -4.61667916, -0.69166075],
           [-0.39675353, -2.62563561, -4.82528105, -6.0607449 , -0.39675353]]), np.array([[ True, False,  True, False,  True],
           [False,  True, False,  True,  True],
           [False, False,  True, False, False]], dtype=bool), np.array([[ 1.06071093,  0.        ,  8.21049603,  0.        ,  1.06071093],
           [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]), np.array([[ 0.50249434,  0.90085595],
           [-0.68372786, -0.12289023],
           [-0.93576943, -0.26788808]]), np.array([[ 0.53035547],
           [-0.69166075],
           [-0.39675353]]), np.array([[-0.7415562 , -0.0126646 , -5.65469333, -0.0126646 , -0.7415562 ]]), np.array([[ 0.32266394,  0.49683389,  0.00348883,  0.49683389,  0.32266394]]), np.array([[-0.6871727 , -0.84520564, -0.67124613]]), np.array([[-0.0126646]]))


    return X_assess, Y_assess, cache
    

**Problem Açıklaması**: Fransız Futbol Şirketi tarafından yapay zeka uzmanı olarak işe alındınız. Fransız takımının oyuncularının daha sonra kafalarıyla vurabilmeleri için Fransa'nın kalecisinin topa vurması gereken pozisyonları önermenizi istiyorlar.
<img src="https://drive.google.com/uc?id=1x_C6I9yi91gJUrUxX_-VQ-9P-ipK4S7U" style="width:600px;height:350px;" alt="field_kiank" title="field_kiank">

<caption><center> <u> **Şekil 1** </u>: **Futbol sahası**<br> 
Kaleci topu havaya atar, her takımın oyuncuları kafalarıyla topa vurmak için savaşırlar. </center></caption>


Size Fransa'nın son 10 maçından aşağıdaki 2B veri kümesini veriyorlar.

In [ ]:
train_X, train_Y, test_X, test_Y = load_2D_dataset()

Her nokta, Fransız kalecinin futbol sahasının sol tarafından şutunu çekmesinden sonra bir futbolcunun kafasıyla topa vurduğu futbol sahasındaki bir konuma karşılık gelir.
- Nokta mavi ise Fransız oyuncu kafasıyla topa vurmayı başarmış demektir.
- Nokta kırmızı ise diğer takımın oyuncusu topa kafasıyla vurmuş demektir.

**Hedefiniz**: Sahada kalecinin topa vurması gereken pozisyonları bulmak için bir derin öğrenme modeli kullanın.

**Veri kümesinin analizi**: Bu veri kümesi biraz gürültülü, ancak sol üst yarıyı (mavi) sağ alt yarıdan (kırmızı) ayıran çapraz bir çizgi iyi işleyecek gibi görünüyor.

İlk önce düzenli olmayan bir modeli deneyeceksiniz. Ardından, bunu nasıl düzenli hale getireceğinizi öğrenecek ve French Football Corporation'ın sorununu çözmek için hangi modeli seçeceğinize karar vereceksiniz.

## 1 - Düzenlenmemiş (Non-regularized) model

Aşağıdaki sinir ağını kullanacaksınız (aşağıda sizin için zaten uygulanmış). Bu model kullanılabilir:

- *düzenleme modunda (regularization mode)* -- `lambd` girişini sıfır olmayan bir değere ayarlayarak. Python'da "`lambda`" tanımlı bir anahtar kelime olduğundan, "`lambda`" yerine "`lambd`" kullanıyoruz.

- *seyreltme modunda (dropout mode)* -- `keep_prob`u birden küçük bir değere ayarlayarak

Modeli ilk önce herhangi bir düzenleme yapmadan deneyeceksiniz. Ardından, uygulayacaksınız:
- *L2 düzenlileştirme (regularization)* -- fonksiyonlar: "``compute_cost_with_regularization()`" ve "`backward_propagation_with_regularization()`"

- *Seyreltme (Dropout)* -- fonksiyonlar: "`forward_propagation_with_dropout()`" ve "`backward_propagation_with_dropout()`"

Her bölümde, uyguladığınız fonksiyonları çağırması için bu modeli doğru girdilerle çalıştıracaksınız. Modele aşina olmak için aşağıdaki koda bir göz atın.

In [ ]:
def model(X, Y, learning_rate = 0.3, num_iterations = 30000, print_cost = True, lambd = 0, keep_prob = 1):
    """
    Implements a three-layer neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (output size, number of examples)
    learning_rate -- learning rate of the optimization
    num_iterations -- number of iterations of the optimization loop
    print_cost -- If True, print the cost every 10000 iterations
    lambd -- regularization hyperparameter, scalar
    keep_prob - probability of keeping a neuron active during drop-out, scalar.
    
    Returns:
    parameters -- parameters learned by the model. They can then be used to predict.
    """
        
    grads = {}
    costs = []                            # to keep track of the cost
    m = X.shape[1]                        # number of examples
    layers_dims = [X.shape[0], 20, 3, 1]
    
    # Initialize parameters dictionary.
    parameters = initialize_parameters(layers_dims)

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        if keep_prob == 1:
            a3, cache = forward_propagation(X, parameters)
        elif keep_prob < 1:
            a3, cache = forward_propagation_with_dropout(X, parameters, keep_prob)
        
        # Cost function
        if lambd == 0:
            cost = compute_cost(a3, Y)
        else:
            cost = compute_cost_with_regularization(a3, Y, parameters, lambd)
            
        # Backward propagation.
        assert(lambd == 0 or keep_prob == 1)    # it is possible to use both L2 regularization and dropout, 
                                            # but this assignment will only explore one at a time
        if lambd == 0 and keep_prob == 1:
            grads = backward_propagation(X, Y, cache)
        elif lambd != 0:
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)
        
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the loss every 10000 iterations
        if print_cost and i % 10000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
        if print_cost and i % 1000 == 0:
            costs.append(cost)
    
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (x1,000)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Modeli herhangi bir düzenleme yapmadan eğitelim ve eğitim/test kümelerinde doğruluğunu gözlemleyelim.

In [ ]:
parameters = model(train_X, train_Y)
print("On the training set:")
predictions_train = predict(train_X, train_Y, parameters)
print("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Eğitim doğruluğu %94,8, test doğruluğu ise %91.5'tir. 
Bu, **temel modeldir** (düzenlemenin bu model üzerindeki etkisini gözlemleyeceksiniz). 
Modelinizin karar sınırını çizmek için aşağıdaki kodu çalıştırın.

In [ ]:
plt.title("Model without regularization")
axes = plt.gca()
axes.set_xlim([-0.75, 0.40])
axes.set_ylim([-0.75, 0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

Düzenlenmemiş model açıkça eğitim kümesine ağırı uyuyor. 
Gürültülü noktalara uyuyor! 
Şimdi aşırı uymayı azaltmak için iki tekniğe bakalım.

## 2 - L2 Düzenlemesi (L2 Regularization)

Aşırı uyumu önlemenin standart yolu **L2 düzenleme** olarak adlandırılır. Maliyet fonksiyonunuzu aşağıdakilerden uygun şekilde değiştirmekten oluşur:

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$
'dan

$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} \tag{2}$$
'na

Maliyetinizi değiştirelim ve sonuçlarını gözlemleyelim.

**Alıştırma**: Formül (2) ile verilen maliyeti hesaplayan `compute_cost_with_regularization()` öğesini uygulayın. $\sum\limits_k\sum\limits_j W_{k,j}^{[l]2}$ hesaplamak için şunu kullanın:
```python
np.sum(np.square(Wl))
```
Bunu $W^{[1]}$, $W^{[2]}$ ve $W^{[3]}$ için yapmanız, ardından üç terimi toplamanız ve $ \frac{1}{m} \frac{\lambda}{2} $.

In [ ]:
# GRADED FUNCTION: compute_cost_with_regularization

def compute_cost_with_regularization(A3, Y, parameters, lambd):
    """
    Implement the cost function with L2 regularization. See formula (2) above.
    
    Arguments:
    A3 -- post-activation, output of forward propagation, of shape (output size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    parameters -- python dictionary containing parameters of the model
    
    Returns:
    cost - value of the regularized loss function (formula (2))
    """
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    
    cross_entropy_cost = compute_cost(A3, Y) # This gives you the cross-entropy part of the cost
    
    ### START CODE HERE ### (approx. 1 line)
    L2_regularization_cost = lambd * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3))) / (2 * m)
    ### END CODER HERE ###
    
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost

In [ ]:
A3, Y_assess, parameters = compute_cost_with_regularization_test_case()

print("cost = " + str(compute_cost_with_regularization(A3, Y_assess, parameters, lambd = 0.1)))

Elbette maliyeti değiştirdiğiniz için geriye doğru yayılmayı da değiştirmelisiniz! Tüm gradyanlar bu yeni maliyete göre hesaplanmalıdır.

**Alıştırma**: Düzenlemeyi hesaba katmak için geriye doğru yayılmada gereken değişiklikleri uygulayın. Değişiklikler yalnızca dW1, dW2 ve dW3 ile ilgilidir. Her biri için, düzenlileştirme teriminin gradyanını ($\frac{d}{dW} ( \frac{1}{2}\frac{\lambda}{m}  W^2) = \frac{\lambda}{m} W$) eklemelisiniz.

In [ ]:
# GRADED FUNCTION: backward_propagation_with_regularization

def backward_propagation_with_regularization(X, Y, cache, lambd):
    """
    Implements the backward propagation of our baseline model to which we added an L2 regularization.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation()
    lambd -- regularization hyperparameter, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    
    ### START CODE HERE ### (approx. 1 line)
    dW3 = 1. / m * np.dot(dZ3, A2.T) + (lambd * W3) / m
    ### END CODE HERE ###
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    ### START CODE HERE ### (approx. 1 line)
    dW2 = 1. / m * np.dot(dZ2, A1.T) + (lambd * W2) / m
    ### END CODE HERE ###
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    ### START CODE HERE ### (approx. 1 line)
    dW1 = 1. / m * np.dot(dZ1, X.T) + (lambd * W1) / m
    ### END CODE HERE ###
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3, "dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
X_assess, Y_assess, cache = backward_propagation_with_regularization_test_case()

grads = backward_propagation_with_regularization(X_assess, Y_assess, cache, lambd=0.7)
print ("dW1 = " + str(grads["dW1"]))
print ("dW2 = " + str(grads["dW2"]))
print ("dW3 = " + str(grads["dW3"]))

Şimdi modeli L2 düzenlileştirme $(\lambda = 0.7)$ ile çalıştıralım. `model()` fonksiyonu şunu çağırır:
- `compute_cost` yerine `compute_cost_with_regularization`
- `backward_propagation` yerine `backward_propagation_with_regularization`

In [ ]:
parameters = model(train_X, train_Y, lambd=0.7)
print("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Tebrikler, test kümesindeki doğruluğu %93'e yükseldi. Fransız futbol takımını kurtardın!

Artık eğitim verilerine gereğinden aşırı uyum göstermiyorsunuz. 

Karar sınırını çizelim.

In [ ]:
plt.title("Model with L2-regularization")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Gözlemler**:
- $\lambda$ değeri, bir geliştirme kümesi kullanarak ayarlayabileceğiniz bir hiperparametredir.
- L2 düzenlileştirmesi, karar sınırınızı daha yumuşak hale getirir. $\lambda$ çok büyükse, yüksek sapmalı bir modelle sonuçlanan "aşırı düzleştirme" de mümkündür.

**L2-düzenlileştirme aslında ne yapıyor?**:

L2-düzenlileştirme, küçük ağırlıklı bir modelin büyük ağırlıklı bir modelden daha basit olduğu varsayımına dayanır. Böylece maliyet fonksiyonunda ağırlıkların kare değerlerini cezalandırarak tüm ağırlıkları daha küçük değerlere yönlendirirsiniz. Büyük ağırlıklara sahip olmak maliyet için çok maliyetli hale geliyor! Bu, girdi değiştikçe çıktının daha yavaş değiştiği daha düzgün bir modele yol açar.

<font color='blue'>
**Hatırlamanız gerekenler** -- L2 düzenlileştirmesinin aşağıdakiler üzerindeki etkileri:

- Maliyet hesabı:
  - Maliyete bir düzenleme süresi eklendi

- Geri yayılım işlevi:
  - Ağırlık matrislerine göre gradyanlarda ekstra terimler vardır.

- Ağırlıklar küçülür ("ağırlık azalması"):
  - Ağırlıklar daha küçük değerlere güncellenir.

## 3 - Seyreltme (Dropout)

Son olarak, **seyreltme (dropout)**, derin öğrenmeye özgü, yaygın olarak kullanılan bir düzenlileştirme tekniğidir.
**Her iterasyonda bazı nöronları rastgele kapatır.** Bunun ne anlama geldiğini görmek için bu iki videoyu izleyin!

In [ ]:
%%html
<video height="720" src="https://drive.google.com/uc?id=1iUUt-0i7UfGW4gmZTVkq0VIA73uXrDn_" type="video/mp4" controls>
</video>

<h3>Şekil2: İkinci gizli katmanda seyreltme</h3>
<p>
  At each iteration, you shut down (= set to zero) each neuron of a layer with probability 1−keep_prob or keep it with probability keep_prob (50% here). The dropped neurons don't contribute to the training in both the forward and backward propagations of the iteration.
</p>

In [ ]:
%%html
<video height="720" src="https://drive.google.com/uc?id=1AnssB6AZ8iasDDrfNzLp7_PjT67eAjYN" type="video/mp4" controls>
</video>
<h3>Şekil3: Birinci ve üçüncü gizli katmanlarda seyreltme</h3>
<p>
1st layer: we shut down on average 40% of the neurons. 3rd layer: we shut down on average 20% of the neurons. 
</p>

Bazı nöronları kapattığınızda, aslında modelinizi değiştirirsiniz. Seyreltmenin arkasındaki fikir, her yinelemede, nöronlarınızın yalnızca bir alt kümesini kullanan farklı bir model eğitmenizdir. Seyreltme ile, nöronlarınız diğer bir spesifik nöronun aktivasyonuna karşı daha az duyarlı hale gelir, çünkü bu diğer nöron herhangi bir zamanda kapatılabilir.

### 3.1 - Seyreltme ile ileriye yayılma

**Alıştırma**: Seyreltme ile ileriye yayılımı uygulayın. 3 katmanlı bir sinir ağı kullanıyorsunuz ve birinci ve ikinci gizli katmanlara dropout ekleyeceksiniz. Giriş katmanına veya çıkış katmanına dropout uygulamayacağız.

**Talimatlar**:
Birinci ve ikinci katmanlardaki bazı nöronları kapatmak istiyorsunuz. Bunu yapmak için 4 Adımı gerçekleştireceksiniz:

1. Derste, `np.random.rand()` kullanarak $a^{[1]}$ ile aynı şekle sahip bir $d^{[1]}$ değişkeni yaratmayı tartışmıştık. 1. Burada, vektörleştirilmiş bir uygulama kullanacaksınız, bu nedenle rastgele bir matris oluşturun $D^{[1]} = [d^{[1](1)} d^{[1](2)} ... d^{[1](m)}] $ ile aynı boyutta $A^{[1]}$.

2. $D^{[1]}$ içindeki değerleri eşikleyerek, $D^{[1]}$ öğesinin her girişini 0 olasılıkla (`1-keep_prob`) veya 1 olasılıkla (`keep_prob`) olacak şekilde ayarlayın. uygun şekilde. İpucu: X matrisinin tüm girişlerini 0'a (giriş 0,5'ten küçükse) veya 1'e (giriş 0,5'ten büyükse) ayarlamak için şunları yaparsınız: 'X = (X < 0,5)'. 0 ve 1'in sırasıyla Yanlış ve Doğru'ya eşdeğer olduğuna dikkat edin.

3. $A^{[1]}$ öğesini $A^{[1]} * D^{[1]}$ olarak ayarlayın. (Bazı nöronları kapatıyorsunuz). $D^{[1]}$'ı bir maske olarak düşünebilirsiniz, böylece başka bir matrisle çarpıldığında bazı değerleri kapatır.

4. $A^{[1]}$'ı `keep_prob` ile bölün. Bunu yaparak, maliyetin sonucunun, seyreltme olmadan aynı beklenen değere sahip olacağını garanti ediyorsunuz. (Bu tekniğe seyreltmenin tersi (inverted dropout) de denir.)

In [ ]:
# GRADED FUNCTION: forward_propagation_with_dropout

def forward_propagation_with_dropout(X, parameters, keep_prob=0.5):
    """
    Implements the forward propagation: LINEAR -> RELU + DROPOUT -> LINEAR -> RELU + DROPOUT -> LINEAR -> SIGMOID.
    
    Arguments:
    X -- input dataset, of shape (2, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (20, 2)
                    b1 -- bias vector of shape (20, 1)
                    W2 -- weight matrix of shape (3, 20)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    keep_prob - probability of keeping a neuron active during drop-out, scalar
    
    Returns:
    A3 -- last activation value, output of the forward propagation, of shape (1,1)
    cache -- tuple, information stored for computing the backward propagation
    """
    
    np.random.seed(1)
    
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    ### START CODE HERE ### (approx. 4 lines)         # Steps 1-4 below correspond to the Steps 1-4 described above. 
    D1 = np.random.rand(A1.shape[0], A1.shape[1])     # Step 1: initialize matrix D1 = np.random.rand(..., ...)
    D1 = D1 < keep_prob                            # Step 2: convert entries of D1 to 0 or 1 (using keep_prob as the threshold)
    A1 = A1 * D1                                      # Step 3: shut down some neurons of A1
    A1 = A1 / keep_prob                               # Step 4: scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    ### START CODE HERE ### (approx. 4 lines)
    D2 = np.random.rand(A2.shape[0], A2.shape[1])     # Step 1: initialize matrix D2 = np.random.rand(..., ...)
    D2 = D2 < keep_prob                           # Step 2: convert entries of D2 to 0 or 1 (using keep_prob as the threshold)                           
    A2 = A2 * D2                                      # Step 3: shut down some neurons of A2
    A2 = A2 / keep_prob                               # Step 4: scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3)
    
    return A3, cache

In [ ]:
X_assess, parameters = forward_propagation_with_dropout_test_case()

A3, cache = forward_propagation_with_dropout(X_assess, parameters, keep_prob=0.7)
print ("A3 = " + str(A3))

### 3.2 - Seyreltme ile geriye doğru yayılma

**Alıştırma**: Seyreltme ile geriye doğru yayılımı uygulayın. Daha önce olduğu gibi, 3 katmanlı bir ağ eğitimi alıyorsunuz. Önbellekte depolanan $D^{[1]}$ ve $D^{[2]}$ maskelerini kullanarak birinci ve ikinci gizli katmanlara dropout ekleyin.

**Talimat**:
Seyreltme ile geri yayılım aslında oldukça kolaydır. 2 Adımı gerçekleştirmeniz gerekecek:

1. Daha önce `A1`e $D^{[1]}$ maskesi uygulayarak ileri yayılma sırasında bazı nöronları kapatmıştınız. Geri yayılımda, aynı $D^{[1]}$ maskesini `dA1`"e yeniden uygulayarak aynı nöronları kapatmanız gerekecektir.

2. İleri yayılım sırasında `A1`i `keep_prob` ile böldünüz. Geri yayılımda, bu nedenle, `dA1`i tekrar `keep_prob` ile bölmeniz gerekir (hesap yorumu şu şekildedir: $A^{[1]}$, `keep_prob` ile ölçeklenirse, türevi $dA^{[1 ]}$ aynı `keep_prob` tarafından da ölçeklenir).

In [ ]:
# GRADED FUNCTION: backward_propagation_with_dropout

def backward_propagation_with_dropout(X, Y, cache, keep_prob):
    """
    Implements the backward propagation of our baseline model to which we added dropout.
    
    Arguments:
    X -- input dataset, of shape (2, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation_with_dropout()
    keep_prob - probability of keeping a neuron active during drop-out, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    dA2 = np.dot(W3.T, dZ3)
    ### START CODE HERE ### (≈ 2 lines of code)
    dA2 = dA2 * D2              # Step 1: Apply mask D2 to shut down the same neurons as during the forward propagation
    dA2 = dA2 / keep_prob              # Step 2: Scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T)
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    ### START CODE HERE ### (≈ 2 lines of code)
    dA1 = dA1 * D1              # Step 1: Apply mask D1 to shut down the same neurons as during the forward propagation
    dA1 = dA1 / keep_prob              # Step 2: Scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
X_assess, Y_assess, cache = backward_propagation_with_dropout_test_case()

gradients = backward_propagation_with_dropout(X_assess, Y_assess, cache, keep_prob=0.8)

print ("dA1 = " + str(gradients["dA1"]))
print ("dA2 = " + str(gradients["dA2"]))

Şimdi modeli bırakma ile çalıştıralım (`keep_prob = 0.86`). Bu, her iterasyonda katman 1 ve 2'nin her nöronunu %24 olasılıkla kapattığınız anlamına gelir. 'model()' işlevi şimdi şunu çağıracak:
- `forward_propagation` yerine `forward_propagation_with_dropout`.
- `backward_propagation`yerine `backward_propagation_with_dropout`.

In [ ]:
parameters = model(train_X, train_Y, keep_prob=0.86, learning_rate=0.3)

print("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Seyreltme(dropout) harika çalışıyor! Test doğruluğu tekrar arttı (%95'e kadar)! Modeliniz eğitim kümesine aşırı uyum göstermiyor ve test kümesinde harika bir iş çıkarıyor. Fransız futbol takımı sana sonsuza kadar minnettar kalacak!

Karar sınırını çizmek için aşağıdaki kodu çalıştırın.

In [ ]:
plt.title("Model with dropout")
axes = plt.gca()
axes.set_xlim([-0.75, 0.40])
axes.set_ylim([-0.75, 0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Not**:
- Seyreltmeyi kullanırken yapılan **yaygın bir hata** onu hem eğitimde hem de testte kullanmaktır. Seyreltmeyi (düğümleri rastgele ortadan kaldırın) yalnızca eğitimde kullanmalısınız.
- [tensorflow](https://www.tensorflow.org/api_docs/python/tf/nn/dropout), [PaddlePaddle](http://doc.paddlepaddle.org/release_doc/0.9.0/doc/ui/api/trainer_config_helpers/attrs.html), [keras](https://keras.io/layers/core/#dropout) veya [caffe](http://caffe.berkeleyvision.org/tutorial/layers/dropout.html) derin öğrenme kütüphanelerinde dropout birlikte gelir. Stres yapmayın - yakında bu çerçevelerden bazılarını öğreneceksiniz.

<font color='blue'>
**Seyreltme (dropout) hakkında hatırlamanız gerekenler:**

- Seyreltme bir düzenlileştirme tekniğidir.

- Seyreltmeyi yalnızca eğitim sırasında kullanırsınız. Test süresi boyunca seyreltme kullanmayın (düğümleri rastgele ortadan kaldırın).

- Hem ileri hem de geri yayılım sırasında seyreltme uygulayın.

- Eğitim süresi boyunca, aktivasyonlar için aynı beklenen değeri korumak için her seyreltme katmanını keep_prob'a bölün. Örneğin, keep_prob 0,5 ise, ortalama olarak düğümlerin yarısını kapatacağız, bu nedenle çıktı, yalnızca kalan yarısı çözüme katkıda bulunduğundan 0,5 ile ölçeklenecektir. 0,5'e bölmek, 2 ile çarpmaya eşdeğerdir. Bu nedenle, çıktı artık aynı beklenen değere sahiptir. Keep_prob 0,5'ten farklı değerler olduğunda bile bunun çalışıp çalışmadığını kontrol edebilirsiniz.

## 4. Sonuçlar

**İşte üç modelimizin sonuçları**:

<table> 
    <tr>
        <td>
        model
        </td>
        <td>
        train accuracy
        </td>
        <td>
        test accuracy
        </td>
    </tr>
        <td>
        3-layer NN without regularization
        </td>
        <td>
        95%
        </td>
        <td>
        91.5%
        </td>
    <tr>
        <td>
        3-layer NN with L2-regularization
        </td>
        <td>
        94%
        </td>
        <td>
        93%
        </td>
    </tr>
    <tr>
        <td>
        3-layer NN with dropout
        </td>
        <td>
        93%
        </td>
        <td>
        95%
        </td>
    </tr>
</table> 

Düzenlileştirmenin eğitim kümesi performansına zarar verdiğini unutmayın! Bunun nedeni, ağın eğitim kümesine aşırı uyum yeteneğini sınırlamasıdır. Ancak sonuçta daha iyi test doğruluğu sağladığı için sisteminize yardımcı oluyor.

Bu görevi tamamladığınız için tebrikler! Ve ayrıca Fransız futbolunda devrim yarattığınız için. :-)

<font color='blue'>
**Bu çalışma dosyasında hatırlamanızı istediğimiz şey**:

- Düzenlileştirme, aşırı uyumu azaltmanıza yardımcı olacaktır.
- Düzenlileştirme, ağırlıklarınızı daha düşük değerlere çekecektir.
- L2 düzenlileştirme (Regularizatio) ve Seyreltme (Dropout), çok etkili iki düzenlileştirme tekniğidir.